In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
car = pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")

In [ ]:
car.head()

In [ ]:
car.shape
car.info()
car.describe()

In [ ]:
#Categorical features
print(car["Seller_Type"].unique())
print(car["Transmission"].unique())
print(car["Fuel_Type"].unique())

In [ ]:
#Missing values
car.isnull().sum()

In [ ]:
car.columns

In [ ]:
final_data=car[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven','Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]
final_data.head()

In [ ]:
#create new feature using year
final_data["no_years"]=2020-final_data["Year"]
final_data.drop("Year",axis=1,inplace=True)

In [ ]:
final_data.head()

In [ ]:
final_data=pd.get_dummies(final_data,drop_first=True)

In [ ]:
final_data.head()

In [ ]:
final_data.corr()

In [ ]:

corrmat=final_data.corr()
top_corr=corrmat.index

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(final_data[top_corr].corr(),annot=True,cmap='RdYlGn')
plt.show()

In [ ]:
X=final_data.iloc[:,1:]
y=final_data.iloc[:,0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor()
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
#print(n_estimators)

from sklearn.model_selection import RandomizedSearchCV
#Randomized Search CV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_   #finding best parameters

In [ ]:
rf_random.best_score_   #best score by parameters

In [ ]:
prediction=rf_random.predict(X_test)
prediction

In [ ]:
sns.distplot(y_test-prediction)  #Visualizing Error 

In [ ]:
plt.scatter(y_test,prediction)   #It shows how our y_test and predictions are related

In [ ]:
#observe the Error metrics
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, prediction))
print('MSE:', metrics.mean_squared_error(y_test, prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))